In [20]:
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import fix_yahoo_finance as yf

In [22]:
# doanload data from yahoo finanance 
yf.pdr_override()
# Doanload data frame
AAPL = pdr.get_data_yahoo('AAPL',start = '2010-01-01', end='2017-12-31')
AAPL

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2010-01-04,30.49,30.64,30.34,30.57,20.31,123432400
2010-01-05,30.66,30.80,30.46,30.63,20.34,150476200
2010-01-06,30.63,30.75,30.11,30.14,20.02,138040000
2010-01-07,30.25,30.29,29.86,30.08,19.98,119282800
2010-01-08,30.04,30.29,29.87,30.28,20.12,111902700
2010-01-11,30.40,30.43,29.78,30.02,19.94,115557400
2010-01-12,29.88,29.97,29.49,29.67,19.71,148614900
2010-01-13,29.70,30.13,29.16,30.09,19.99,151473000
2010-01-14,30.02,30.07,29.86,29.92,19.87,108223500
